# Interrupción Legal del Embarazo

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' #####SÓLO TRAE INFO 2016
FILE_NAME = 'interrupcion_embarazo'

## Import

In [3]:
from mariachis.models import InterrupcionEmbarazo
inem = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)
print(inem)

Directorio: 	/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


In [4]:
df = inem.full_import(api=False)
df.sample()

Archivo con nombre interrupcion_embarazo.csv fue encontrado en /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
40977,2020.0,DICIEMBRE,2020-12-21,SI,SEPARADA,29.0,NINGUNO,PREPARATORIA,NaN,CATOLICA,...,NO,SI,NINGUNO,NaN,NaN,NaN,NaN,DFSSA004270,MEDICAMENTO,NaN


## Dict

In [5]:
# df[df['sememb']>12].to_csv('test.csv', index=False)
# df['sememb'].describe([i/10 for i in range(10)])

In [6]:
# Diccionario para aplicar limpieza por tipo de variable
full_dict = {}

# Variables que o son 0 o > 0
full_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Numéricas que serán partidas en rangos
full_dict['vars_num'] = {
    'edad': [9,11,15,18,21,25,29,35],
    'menarca': [7,11,12,13,14,15],
    'fsexual': [7,11,15,18,21],
    'sememb': [3,6,9,12,20],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
full_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
full_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Sólo cambiar nulos por "DESCONOCIDO"
full_dict['vars_just_fill_na'] = ['entidad', 'motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep']

# Agrupar categorías
full_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA',
        'MAESTRIA':'POSGRADO',
        'DOCTORADO':'POSGRADO',
        'PREPARATORIA':'PREPARATORIA',
        'SECUNDARIA':'SECUNDARIA',
        'PRIMARIA':'PRIMARIA',
        'SIN ACCESO A LA EDUCACION FORMAL':'SIN_ACCESO',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

In [7]:
from pandas import qcut
from IPython.display import display

# Cambiar a False si NO se desea conocer los rangos propuestos
show_default_ranges = True

if show_default_ranges:
    aux_dict = {}
    for col in full_dict['vars_num']:
        new_col = df[col].map(inem.clean_number).astype(float)
        orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
        aux_dict[col] = orig_bins[:-1]
    display(aux_dict)

{'edad': array([11., 20., 22., 25., 28., 32.]),
 'menarca': array([ 0., 11., 12., 13., 14.]),
 'fsexual': array([ 0., 15., 16., 17., 18., 19.]),
 'sememb': array([0., 5., 6., 7., 8., 9.]),
 'nhijos': array([0., 1., 2.]),
 'gesta': array([0., 1., 2., 3., 4.]),
 'c_num': array([0., 1., 2.]),
 'p_semgest': array([0., 5., 6., 7., 8., 9.]),
 'p_diasgesta': array([0., 1., 2., 3., 4., 5.])}

## Transform

In [8]:
# Quitar comentarios para ejecutar proceso. Nota: Demoró 13min con 77K registros
# X = al.wrangling_aborto(df, full_dict, export_result=True)
# X.sample(4)

## Checkpoint

In [9]:
from pandas import read_csv
from pathlib import Path

# Si no se desea re-clusterizar, entonces mantener comentarios de la sección pasada e importar el resultado desde esta celda
vars_cat = list(set(
            full_dict['vars_first_word']+
            list(full_dict['vars_cat'].keys())+
            full_dict['vars_yes_no']+
            full_dict['vars_just_fill_na']
        ))
cluster_cols = vars_cat+[f'rango_{col}' for col in full_dict['vars_num'].keys()]+full_dict['vars_numbin']
add_cols = ['año','mes','alc_o_municipio']

df = read_csv(Path(BASE_DIR).joinpath(FILE_NAME+'_cluster.csv'), low_memory=False)
df.sample()

,index,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,...,rango_edad,rango_menarca,rango_fsexual,rango_sememb,rango_nhijos,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,cluster
24951,26580,2019.0,MARZO,2019-03-15,DESCONOCIDO,SOLTERX,34.0,NINGUNO,PREPARATORIA O MENOR,EMPLEADX,...,> 32,12,01 a 15,06,DESCONOCIDO,> 04,02,06,> 05,C


## Perfilamiento

In [10]:
inem.profiles(df[cluster_cols+add_cols+['cluster']])

cluster,A,B,C,D,E
año,2017,2017,2019,2017,2017
